In [13]:
eggnog="eggnog/test_Chroococcidiopsis_sp._CCMEE_29_(cyanobacteria)_GCF_023558375.1.faa.emapper.annotations"
interpro="interpro/Chroococcidiopsis_sp._CCMEE_29_(cyanobacteria)_GCF_023558375.1.faa.tsv"
refseq="annotati_Refseq/Chroococcidiopsis_sp._CCMEE_29_(cyanobacteria)_GCF_023558375.gbff"

In [41]:
import pandas as pd
eggnog_df = pd.read_csv(eggnog, sep='\t', header=4,skipfooter=3, engine='python')
eggnog_df
#new dataframe with only columns #query,GOs,KEGG_ko
eggnog_df = eggnog_df[['#query', 'GOs', 'KEGG_ko']]
eggnog_df
#rename GOs to GOs_eggnog and KEGG_ko to ko_eggnog
eggnog_df = eggnog_df.rename(columns={'GOs': 'GOs_eggnog', 'KEGG_ko': 'ko_eggnog', '#query': 'protein_id'})
#set index to protein_id
eggnog_df = eggnog_df.set_index('protein_id')
eggnog_df

,GOs_eggnog,ko_eggnog
protein_id,,
WP_009632395.1,-,ko:K04751
WP_041548398.1,"GO:0003674,GO:0005488,GO:0005575,GO:0005622,GO...",ko:K02518
WP_073591590.1,-,ko:K08696
WP_190736661.1,-,-
WP_250121152.1,-,-
...,...,...
WP_275983393.1,-,-
WP_275983394.1,"GO:0003674,GO:0005488,GO:0005515,GO:0005575,GO...",ko:K02708
WP_275983395.1,-,ko:K02003


In [78]:
interpro_df = pd.read_csv(interpro, sep='\t', header=None)
interpro_df
interpro_df = interpro_df.rename(columns={0: 'protein_id', 13: 'GOs_interpro'})
#drop duplicates of protein_id
interpro_df = interpro_df.drop_duplicates(subset='protein_id')
#new dataframe with only columns #query,GOs_interpro
interpro_df = interpro_df[['protein_id', 'GOs_interpro']]

#set index to protein_id
interpro_df = interpro_df.set_index('protein_id')
interpro_df

,GOs_interpro
protein_id,
WP_250121481.1,GO:0005829(PANTHER)
WP_250124019.1,GO:0004775(PANTHER)|GO:0004776(PANTHER)|GO:000...
WP_250124642.1,-
WP_250121995.1,GO:0015979(InterPro)|GO:0015995(InterPro)|GO:0...
WP_250124302.1,-
...,...
WP_250122548.1,-
WP_250123271.1,-
WP_250125842.1,-


In [74]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
#parse refseq file
refseq_record = SeqIO.parse(refseq, "genbank")
#extract for every CDS the GO_process, GO_function and GO_component
refseq_df = pd.DataFrame(columns=['GO_process', 'GO_component', 'GO_function', 'protein_id', 'gene'])
for record in refseq_record:
    for feature in record.features:
        if feature.type == "CDS":
            #get the GO_process, GO_function and GO_component, protein_id and gene
            GO_process = feature.qualifiers.get('GO_process')
            GO_component = feature.qualifiers.get('GO_component')
            GO_function = feature.qualifiers.get('GO_function')
            protein_id = feature.qualifiers.get('protein_id', ['None'])[0]
            gene = feature.qualifiers.get('gene', ['None'])[0]
            #new dataframe with protein_id as row index
            if protein_id !='None':
                data = pd.DataFrame({'GO_process': [GO_process], 'GO_component': [GO_component], 'GO_function': [GO_function], 'protein_id': [protein_id], 'gene': [gene]})
                refseq_df = pd.concat([refseq_df, data])
refseq_df = refseq_df.set_index('protein_id')


In [53]:
refseq_df

,GO_process,GO_component,GO_function,gene
protein_id,,,,
WP_275983370.1,None,None,None,None
WP_250123595.1,None,None,None,None
WP_250123596.1,[GO:0000162 - tryptophan biosynthetic process ...,None,[GO:0004049 - anthranilate synthase activity [...,None
WP_250123597.1,[GO:0006568 - tryptophan metabolic process [Ev...,None,[GO:0004425 - indole-3-glycerol-phosphate synt...,trpC
WP_250123598.1,[GO:0000162 - tryptophan biosynthetic process ...,None,[GO:0004834 - tryptophan synthase activity [Ev...,trpA
...,...,...,...,...
WP_250126520.1,None,None,None,None
None,None,None,[GO:0004803 - transposase activity [Evidence I...,None
WP_250126521.1,None,None,None,None


In [81]:
df = eggnog_df.join(refseq_df, how='inner')
df = df.join(interpro_df, how='inner')
df

,GOs_eggnog,ko_eggnog,GO_process,GO_component,GO_function,gene,GOs_interpro
protein_id,,,,,,,
WP_009632395.1,-,ko:K04751,None,None,None,None,GO:0006808(InterPro)|GO:0030234(InterPro)
WP_041548398.1,"GO:0003674,GO:0005488,GO:0005575,GO:0005622,GO...",ko:K02518,[GO:0006413 - translational initiation [Eviden...,None,[GO:0003743 - translation initiation factor ac...,infA,GO:0003723(InterPro)|GO:0003743(InterPro)|GO:0...
WP_073591590.1,-,ko:K08696,None,None,None,None,GO:0031469(InterPro)
WP_190736661.1,-,-,None,None,None,None,-
WP_250121152.1,-,-,None,None,[GO:0004803 - transposase activity [Evidence I...,None,-
...,...,...,...,...,...,...,...
WP_256478997.1,-,ko:K07480,"[GO:0006313 - transposition, DNA-mediated [Evi...",None,"[GO:0003677 - DNA binding [Evidence IEA], GO:0...",None,GO:0003677(InterPro)|GO:0004803(InterPro)|GO:0...
WP_256478998.1,-,ko:K07480,"[GO:0006313 - transposition, DNA-mediated [Evi...",None,"[GO:0003677 - DNA binding [Evidence IEA], GO:0...",None,GO:0003677(InterPro)|GO:0004803(InterPro)|GO:0...
WP_256479001.1,-,ko:K07494,None,None,None,None,-


KeyError: '[1] not found in axis'

In [88]:
import networkx
import obonet

# Load the GO graph
url = "/home/davide/Downloads/go-basic.obo"
graph = obonet.read_obo(url)

# Create a dictionary with id as key and name and namespace as value
dict = {id_: (data.get('name'), data.get('namespace')) 
                          for id_, data in graph.nodes(data=True)}

names="""GO:0003723(InterPro)|GO:0003743(InterPro)|GO:0006413(InterPro)"""
names = names.split('|')
names = [name.split('(')[0] for name in names]
for name in names:
    print(name, dict.get(name, ['Not found']))
print(dict.get('GO:0006313', ['Not found']))


GO:0003723 ('RNA binding', 'molecular_function')
GO:0003743 ('translation initiation factor activity', 'molecular_function')
GO:0006413 ('translational initiation', 'biological_process')
('DNA transposition', 'biological_process')
